# Data Update (Run For Each GW)

This notebook updates the training data weekly for the gameweek points prediction model. The notebook takes in new PL stats per gameweek and adds them as new rows to the original training dataset.

### REMINDERS BEFORE RUNNING:
- Download updated player_stats_2223 (https://github.com/vaastav/Fantasy-Premier-League/tree/master/data/2022-23)
- Download updated team_standard_stats_2223 & player_standard_stats_2223 (https://fbref.com/en/comps/9/stats/Premier-League-Stats)

In [1]:
#Import relevant libraries and packages
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#Paths
path = Path('Data')
path_20_21 = Path('Data/2020-21')
path_21_22 = Path('Data/2021-22')
path_22_23 = Path('Data/2022-23')

#Import data sets
#Training data
training_data = pd.read_csv(path/'training_data_updated.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})
training_data = training_data.reset_index()

#22-23 season gameweeks
season_gws = pd.read_csv(path/'remaining_season.csv', index_col=0)

#Player info 21-22 and 22-23
player_stats_2122 = pd.read_csv(path_21_22/'gws/merged_gw.csv')
player_stats_2223 = pd.read_csv(path_22_23/'gws/merged_gw.csv')

#Team stats 20-21, 21-22 and 22-23
team_standard_stats_2021 = pd.read_csv(path_20_21/'team_standard_stats_2021.csv')
team_standard_stats_2122 = pd.read_csv(path_21_22/'team_standard_stats_2122.csv')
team_standard_stats_2223 = pd.read_csv(path_22_23/'team_standard_stats_2223.csv')

#Player stats 21-22 and 22-23
player_standard_stats_2122 = pd.read_csv(path_21_22/'player_standard_stats_2122.csv')
player_standard_stats_2223 = pd.read_csv(path_22_23/'player_standard_stats_2223.csv')

#Player's raw data 22-23
players_raw = pd.read_csv(path_22_23/'players_raw.csv')

#22-23 Teams (Teams & IDs)
teams = pd.read_csv(path_22_23/'teams.csv')

#Cleaned player data 22-23
cleaned_players = pd.read_csv(path_22_23/'cleaned_players.csv')

In [2]:
#Function to merge dfs by partial match (substring)
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

We will create a new training data frame with the original training data from the 20-21 season, the 21-22 season (adding gw 38 data), and the 22-23 season (adding each new gw weekly):

## 20-21 Training Data:

In [14]:
# original_training_data_2021 = original_training_data[original_training_data['season'] == '2021'][['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
#        'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
#        'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
#        'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
#        'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]

# original_training_data_2021['team'] = original_training_data_2021['team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# original_training_data_2021['opponent_team'] = original_training_data_2021['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# training_data_2021 = original_training_data_2021.merge(team_standard_stats_2021, on='team')
# training_data_2021 = training_data_2021.sort_values(['gw','team'])
# training_data_2021 = training_data_2021.reset_index()
# training_data_2021 = training_data_2021.drop('index', axis=1)
# training_data_2021

## 21-22 Training Data:

In [15]:
# original_training_data_2122 = original_training_data[original_training_data['season'] == '2122'][['player', 'position', 
#        'gw', 'team', 'opponent_team', 'was_home',
#        'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
#        'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
#        'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
#        'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]
                                                                                                 
# original_training_data_2122['team'] = original_training_data_2122['team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# original_training_data_2122['opponent_team'] = original_training_data_2122['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# training_data_2122 = original_training_data_2122.merge(team_standard_stats_2122, on='team')
# training_data_2122 = training_data_2122.sort_values(['gw','team'])
# training_data_2122 = training_data_2122.reset_index()
# training_data_2122 = training_data_2122.drop('index', axis=1)
# training_data_2122

#### Add Gameweek 38 data:

In [16]:
# #Last season's gameweek 38 data (RUN JUST ONCE) 
# player_stats_2122 = player_stats_2122[player_stats_2122['GW'] == 38]
# player_stats_2122['season'] = '2122'
# relevant_columns = ['name', 'position', 'GW', 'team', 'opponent_team', 'was_home', 'season', 'minutes','total_points',
#                     'assists', 'bonus', 'bps', 'clean_sheets',
#                     'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
#                     'influence', 'penalties_saved', 'red_cards', 'saves', 'threat',
#                     'yellow_cards', 'team_a_score', 'team_h_score']
# player_stats_2122 = player_stats_2122[relevant_columns]
# player_stats_2122 = player_stats_2122.rename(columns={'name': 'player', 'GW':'gw'})

# #Change position from string to int
# def position_assignment(data):
#     if data['position'] == 'GK':
#         return 1
#     if data['position'] == 'DEF':
#         return 2
#     if data['position'] == 'MID':
#         return 3
#     if data['position'] == 'FWD':
#         return 4
    
# player_stats_2122['position'] = player_stats_2122.apply(position_assignment, axis = 1)

# #Change opponent_team from int to string
# def team_assignment(data):
#     if data['opponent_team'] == 1:
#         return 'Arsenal'
#     if data['opponent_team'] == 2:
#         return 'Aston Villa'
#     if data['opponent_team'] == 3:
#         return 'Brentford'
#     if data['opponent_team'] == 4:
#         return 'Brighton'
#     if data['opponent_team'] == 5:
#         return 'Burnley'
#     if data['opponent_team'] == 6:
#         return 'Chelsea'
#     if data['opponent_team'] == 7:
#         return 'Crystal Palace'
#     if data['opponent_team'] == 8:
#         return 'Everton'
#     if data['opponent_team'] == 9:
#         return 'Leicester City'
#     if data['opponent_team'] == 10:
#         return 'Leeds'
#     if data['opponent_team'] == 11:
#         return 'Liverpool'
#     if data['opponent_team'] == 12:
#         return 'Manchester City'
#     if data['opponent_team'] == 13:
#         return 'Manchester Utd'
#     if data['opponent_team'] == 14:
#         return 'Newcastle Utd'
#     if data['opponent_team'] == 15:
#         return 'Norwich'
#     if data['opponent_team'] == 16:
#         return 'Southampton'
#     if data['opponent_team'] == 17:
#         return 'Tottenham'
#     if data['opponent_team'] == 18:
#         return 'Watford'
#     if data['opponent_team'] == 19:
#         return 'West Ham'
#     if data['opponent_team'] == 20:
#         return 'Wolves'
    
# player_stats_2122['opponent_team'] = player_stats_2122.apply(team_assignment, axis = 1)
# gw_38 = player_stats_2122
# gw_38 = gw_38.sort_values(['team'])
# gw_38

In [126]:
# training_data_2122 = pd.concat([training_data_2122, gw_38])
# training_data_2122

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,3,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
1,Albert Sambi Lokonga,3,1,Arsenal,Brentford,False,2122,90,2,0,...,8.0,0,0,2,0.0,0.1,0.0,1.58,0.97,1.42
2,Alexandre Lacazette,4,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
3,Ben White,2,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
4,Bernd Leno,1,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25111,José Malheiro de Sá,1,38,Wolves,Liverpool,False,2122,45,1,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
24774,Luke Cundle,3,38,Wolves,Liverpool,False,2122,0,0,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
24931,Francisco Machado Mota de Castro Trincão,3,38,Wolves,Liverpool,False,2122,1,1,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
25195,Hugo Bueno,2,38,Wolves,Liverpool,False,2122,0,0,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# #Concatenate 20-21 training data to 21-22 training data
# updated_training_data = pd.concat([training_data_2021, training_data_2122])

## Gameweek 22-23 Data:
#### UPDATE GAMEWEEK EVERY WEEK

In [3]:
#REMINDER TO CHANGE GW WEEKLY TO MOST RECENT GW
gameweek = 5

#Player stats for most recent gameweek
player_stats = player_stats_2223[player_stats_2223['GW'] == gameweek]
relevant_columns = ['name', 'minutes','total_points', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'penalties_saved', 'red_cards', 'saves', 'threat',
       'yellow_cards', 'team_a_score', 'team_h_score']
player_stats = player_stats[relevant_columns]
player_stats = player_stats.rename(columns={'name': 'player'})
player_stats

,player,minutes,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,penalties_saved,red_cards,saves,threat,yellow_cards,team_a_score,team_h_score
2347,Nathan Redmond,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,2
2348,Junior Stanislas,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,0,0
2349,Armando Broja,24,1,0,0,1,0,0.0,0,0,2.7,6.4,0,0,0,21.0,0,1,2
2350,Fabian Schär,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,2
2351,Jonny Evans,90,2,0,0,16,0,0.6,1,0,2.7,26.4,0,0,0,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2950,Oliver Skipp,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,1
2951,Ryan Sessegnon,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,1
2952,Ashley Young,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,2
2953,Jeremy Sarmiento Morante,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,2


In [4]:
#Player Raw Data Merged with player's team
players_raw = players_raw[['first_name', 'second_name', 'team_code']]
teams = teams[['code', 'name']]
players_raw = players_raw.merge(teams, left_on = 'team_code', right_on= 'code')
players_raw['player'] = players_raw['first_name'] + ' ' + players_raw['second_name']
players_raw = players_raw.rename(columns={'name': 'team'})
players_raw = players_raw[['player', 'team']]
players_raw['team'] = players_raw['team'].replace({'Man Utd': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves',
                                          'Leicester': 'Leicester City',
                                          'Man City': 'Manchester City',
                                          'Newcastle': 'Newcastle Utd',
                                          "Nott'm Forest": 'Nottingham Forest',
                                          'Spurs': 'Tottenham'})
players_raw

,player,team
0,Cédric Alves Soares,Arsenal
1,Granit Xhaka,Arsenal
2,Mohamed Elneny,Arsenal
3,Rob Holding,Arsenal
4,Thomas Partey,Arsenal
...,...,...
610,Connor Ronan,Wolves
611,Chem Campbell,Wolves
612,Gonçalo Manuel Ganchinho Guedes,Wolves
613,Matheus Luiz Nunes,Wolves


#### UPDATE GAMEWEEK HERE:

In [5]:
#Gameweek Data
season_gws['opponent_team'] = season_gws['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves'})

season_gws['team'] = season_gws['team'].replace({'Manchester United': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves'})

season_gws = season_gws[['gw','team', 'opponent_team', 'was_home', 'season']]
season_gws = season_gws.drop_duplicates()
season_gws = season_gws.reset_index().drop('index', axis=1)

#CHANGE GAMEWEEK HERE
season_gws = season_gws[season_gws['gw'] == 5]
season_gws

,gw,team,opponent_team,was_home,season
40,5,Manchester City,Nottingham Forest,True,2223
41,5,Leicester City,Manchester Utd,True,2223
42,5,Liverpool,Newcastle Utd,True,2223
43,5,Bournemouth,Wolves,True,2223
44,5,West Ham,Tottenham,True,2223
45,5,Leeds,Everton,True,2223
46,5,Southampton,Chelsea,True,2223
47,5,Arsenal,Aston Villa,True,2223
48,5,Fulham,Brighton,True,2223
49,5,Crystal Palace,Brentford,True,2223


In [6]:
#Merge gameweek info with player names
season_player_merge = season_gws.merge(players_raw, on='team')
season_player_merge = season_player_merge[['player', 'gw', 'team', 'opponent_team', 'was_home', 'season']]
season_player_merge

,player,gw,team,opponent_team,was_home,season
0,Kyle Walker,5,Manchester City,Nottingham Forest,True,2223
1,Ilkay Gündogan,5,Manchester City,Nottingham Forest,True,2223
2,Kevin De Bruyne,5,Manchester City,Nottingham Forest,True,2223
3,John Stones,5,Manchester City,Nottingham Forest,True,2223
4,Riyad Mahrez,5,Manchester City,Nottingham Forest,True,2223
...,...,...,...,...,...,...
610,Ben Mee,5,Brentford,Crystal Palace,False,2223
611,Fin Stevens,5,Brentford,Crystal Palace,False,2223
612,Halil Dervişoğlu,5,Brentford,Crystal Palace,False,2223
613,Mikkel Damsgaard,5,Brentford,Crystal Palace,False,2223


In [7]:
#Add player's position
cleaned_players['player'] = cleaned_players['first_name'] + ' ' + cleaned_players['second_name']
cleaned_players = cleaned_players[['player', 'element_type']]
cleaned_players = cleaned_players.rename(columns={'element_type': 'position'})
season_player_merge = season_player_merge.merge(cleaned_players, on='player')

#Convert position from string to int
def position_assignment(data):
    if data['position'] == 'GK':
        return 1
    if data['position'] == 'DEF':
        return 2
    if data['position'] == 'MID':
        return 3
    if data['position'] == 'FWD':
        return 4

season_player_merge['position'] = season_player_merge.apply(position_assignment, axis = 1)

#Ordered and clean df with player gw data
season_player_merge = season_player_merge[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home', 'season']]
season_player_merge = season_player_merge.drop_duplicates()
season_player_merge

,player,position,gw,team,opponent_team,was_home,season
0,Kyle Walker,2,5,Manchester City,Nottingham Forest,True,2223
1,Ilkay Gündogan,3,5,Manchester City,Nottingham Forest,True,2223
2,Kevin De Bruyne,3,5,Manchester City,Nottingham Forest,True,2223
3,John Stones,2,5,Manchester City,Nottingham Forest,True,2223
4,Riyad Mahrez,3,5,Manchester City,Nottingham Forest,True,2223
...,...,...,...,...,...,...,...
612,Ben Mee,2,5,Brentford,Crystal Palace,False,2223
613,Fin Stevens,2,5,Brentford,Crystal Palace,False,2223
614,Halil Dervişoğlu,4,5,Brentford,Crystal Palace,False,2223
615,Mikkel Damsgaard,3,5,Brentford,Crystal Palace,False,2223


In [8]:
#Cleaning player standard stats
player_standard_stats_2223 = player_standard_stats_2223[['Unnamed: 1', 'Per 90 Minutes.1', 
                                                         'Unnamed: 28', 'Unnamed: 30']]
player_standard_stats_2223.columns = player_standard_stats_2223.iloc[0]
player_standard_stats_2223 = player_standard_stats_2223.drop(index=0)
player_standard_stats_2223 = player_standard_stats_2223.rename(columns={'Player': 'player',
                                                                'xG': 'xg',
                                                                'xA': 'xa',
                                                                'npxG': 'npxg'})
player_standard_stats_2223.to_csv(path_22_23/'player_standard_stats_2223.csv', index=False)
player_standard_stats_2223

,player,xg,xa,npxg
1,Brenden Aaronson,0.25,0.07,0.25
2,Che Adams,0.28,0.16,0.28
3,Tyler Adams,0.01,0.02,0.01
4,Tosin Adarabioyo,0,0.01,0
5,Rayan Aït Nouri,0.04,0.2,0.04
...,...,...,...,...
387,Wilfried Zaha,0.66,0.28,0.42
388,Jordan Zemura,0,0.06,0
389,Oleksandr Zinchenko,0.05,0.09,0.05
390,Hakim Ziyech,0.09,0.11,0.09


In [9]:
#Merging last gameweek's player stats with last gameweek's player info and adding players' xg, xa, and npxg
season_gw = fuzzy_merge(season_player_merge, player_stats, 'player', 'player', threshold=91)
season_gw_stats = season_gw.merge(player_stats, left_on = 'matches', right_on = 'player')
season_gw_stats = season_gw_stats.drop(['player_x', 'matches'], axis=1)
season_gw_stats = season_gw_stats.rename(columns={'player_y': 'player'})
season_gw_stats = season_gw_stats[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score']]

season_gw_stats = fuzzy_merge(season_gw_stats, player_standard_stats_2223, 'player', 'player', threshold=91)
season_gw_stats['matches'].replace('', np.nan, inplace=True)
season_gw_no_match = season_gw_stats[season_gw_stats['matches'].isna()]
season_gw_no_match[['xg', 'xa', 'npxg']] = 0
season_gw_no_match = season_gw_no_match.drop('matches', axis=1)
season_gw_stats = season_gw_stats.dropna(subset=['matches'])
season_gw_stats = season_gw_stats.merge(player_standard_stats_2223, left_on = 'matches', right_on = 'player')
season_gw_stats = season_gw_stats.drop(['player_x', 'matches'], axis=1)
season_gw_stats = season_gw_stats.rename(columns={'player_y': 'player'})
season_gw_stats = season_gw_stats[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]
season_gw_stats = pd.concat([season_gw_stats, season_gw_no_match])
season_gw_stats

/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_74041/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_gw_no_match[['xg', 'xa', 'npxg']] = 0
/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_74041/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_gw_no_match[['xg', 'xa', 'npxg']] = 0
/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_74041/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,penalties_saved,red_cards,saves,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg
0,Kyle Walker,2,5,Manchester City,Nottingham Forest,True,2223,90,6,0,...,0,0,0,2.0,0,0,6,0,0.02,0
1,Kevin De Bruyne,3,5,Manchester City,Nottingham Forest,True,2223,21,1,0,...,0,0,0,17.0,0,0,6,0.23,0.41,0.23
2,John Stones,2,5,Manchester City,Nottingham Forest,True,2223,74,10,1,...,0,0,0,2.0,0,0,6,0.04,0.23,0.04
3,Riyad Mahrez,3,5,Manchester City,Nottingham Forest,True,2223,35,4,1,...,0,0,0,28.0,0,0,6,0.25,0.16,0.25
4,Jack Grealish,3,5,Manchester City,Nottingham Forest,True,2223,0,0,0,...,0,0,0,0.0,0,0,6,0.04,0.32,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Dominic Thompson,2,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0,0,0,0.0,0,1,1,0,0,0
596,Mads Bidstrup,3,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0,0,0,0.0,0,1,1,0,0,0
597,Ellery Balcombe,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0,0,0,0.0,0,1,1,0,0,0
600,Thomas Strakosha,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0,0,0,0.0,0,1,1,0,0,0


In [13]:
#Cleaning team standard stats
team_standard_stats_2223 = team_standard_stats_2223[['Unnamed: 0', 'Per 90 Minutes.1', 
                                                         'Unnamed: 25', 'Unnamed: 27']]
team_standard_stats_2223.columns = team_standard_stats_2223.iloc[0]
team_standard_stats_2223 = team_standard_stats_2223.drop(index=0)
team_standard_stats_2223 = team_standard_stats_2223.rename(columns={'Squad': 'team',
                                                                'xG': 'team_xg',
                                                                'xA': 'team_xa',
                                                                'npxG': 'team_npxg'})
team_standard_stats_2223['team'] = team_standard_stats_2223['team'].replace({'Leeds United': 'Leeds',
                                                                             "Nott'ham Forest": 'Nottingham Forest'})
team_standard_stats_2223.to_csv(path_22_23/'team_standard_stats_2223.csv', index=False)
team_standard_stats_2223

,team,team_xg,team_xa,team_npxg
1,Arsenal,2.09,1.34,2.09
2,Aston Villa,0.86,0.74,0.86
3,Bournemouth,0.36,0.25,0.36
4,Brentford,1.56,1.18,1.56
5,Brighton,1.56,0.96,1.25
6,Chelsea,1.36,0.95,1.21
7,Crystal Palace,1.33,0.96,1.16
8,Everton,1.24,0.73,1.24
9,Fulham,1.38,0.51,1.06
10,Leeds,1.49,0.78,1.49


#### UPDATE GAMEWEEK HERE:

In [14]:
#Add team xg, xa, npxg
gw5 = season_gw_stats 
gw5 = gw5.merge(team_standard_stats_2223, left_on = 'team', right_on ='team')
gw5 = gw5[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg',
       'team_xg', 'team_xa', 'team_npxg']]

gw5

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Kyle Walker,2,5,Manchester City,Nottingham Forest,True,2223,90,6,0,...,2.0,0,0,6,0,0.02,0,2.56,1.61,2.41
1,Kevin De Bruyne,3,5,Manchester City,Nottingham Forest,True,2223,21,1,0,...,17.0,0,0,6,0.23,0.41,0.23,2.56,1.61,2.41
2,John Stones,2,5,Manchester City,Nottingham Forest,True,2223,74,10,1,...,2.0,0,0,6,0.04,0.23,0.04,2.56,1.61,2.41
3,Riyad Mahrez,3,5,Manchester City,Nottingham Forest,True,2223,35,4,1,...,28.0,0,0,6,0.25,0.16,0.25,2.56,1.61,2.41
4,Jack Grealish,3,5,Manchester City,Nottingham Forest,True,2223,0,0,0,...,0.0,0,0,6,0.04,0.32,0.04,2.56,1.61,2.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,Dominic Thompson,2,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
600,Mads Bidstrup,3,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
601,Ellery Balcombe,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
602,Thomas Strakosha,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56


In [15]:
#Concatenating new data to training data
updated_training_data = pd.concat([training_data, gw5])
updated_training_data = updated_training_data.reset_index()
updated_training_data = updated_training_data.drop('index', axis=1)
updated_training_data

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,2,1,Arsenal,Fulham,False,2021,90,6,0,...,4.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
1,Alexandre Lacazette,4,1,Arsenal,Fulham,False,2021,86,7,0,...,48.0,0,3,0,0.4,0.2,0.4,1.41,0.96,1.29
2,Bernd Leno,1,1,Arsenal,Fulham,False,2021,90,7,0,...,0.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
3,Bukayo Saka,3,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
4,Calum Chambers,2,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52744,Dominic Thompson,2,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52745,Mads Bidstrup,3,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52746,Ellery Balcombe,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52747,Thomas Strakosha,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56


In [16]:
#Make sure all team names are the same in entire df
updated_training_data.team.unique()

array(['Arsenal', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Leeds', 'Leicester City', 'Liverpool', 'Newcastle Utd',
       'Sheffield Utd', 'Southampton', 'Tottenham', 'West Brom',
       'West Ham', 'Wolves', 'Aston Villa', 'Burnley', 'Manchester City',
       'Manchester Utd', 'Brentford', 'Norwich', 'Watford', 'Bournemouth',
       'Nottingham Forest'], dtype=object)

In [17]:
#Make sure all opponent_team names are the same in entire df
updated_training_data.opponent_team.unique()

array(['Fulham', 'Chelsea', 'Brighton', 'Southampton', 'Tottenham',
       'Arsenal', 'Liverpool', 'West Brom', 'Leeds', 'West Ham', 'Wolves',
       'Crystal Palace', 'Everton', 'Leicester City', 'Newcastle Utd',
       'Sheffield Utd', 'Manchester Utd', 'Burnley', 'Aston Villa',
       'Manchester City', 'Brentford', 'Watford', 'Norwich',
       'Nottingham Forest', 'Bournemouth'], dtype=object)

In [18]:
#Save updated training data 
updated_training_data.to_csv(path/'training_data_updated.csv', index=False)
updated_training_data

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,2,1,Arsenal,Fulham,False,2021,90,6,0,...,4.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
1,Alexandre Lacazette,4,1,Arsenal,Fulham,False,2021,86,7,0,...,48.0,0,3,0,0.4,0.2,0.4,1.41,0.96,1.29
2,Bernd Leno,1,1,Arsenal,Fulham,False,2021,90,7,0,...,0.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
3,Bukayo Saka,3,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
4,Calum Chambers,2,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52744,Dominic Thompson,2,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52745,Mads Bidstrup,3,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52746,Ellery Balcombe,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
52747,Thomas Strakosha,1,5,Brentford,Crystal Palace,False,2223,0,0,0,...,0.0,0,1,1,0,0,0,1.56,1.18,1.56
